# Install all requeriments

In [ ]:
!pip3 install pandas
!pip3 install sqlalchemy
!pip3 install psycopg2  # For PostgreSQL
!pip3 install pymysql   # For MySQL
!pip3 install cx_Oracle # For Oracle 
!pip3 install pyodbc

# Import and conect the data bases 

Conect to source database

In [ ]:
import pyodbc
import pandas as pd

# Define your connection string
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=Rahul\MSSQLSERVER01;"
    "DATABASE=ECC60jkl_HACK;"
    "Trusted_Connection=yes;"
)

# Establish the connection
connection = pyodbc.connect(connection_string)

# Use the connection to query the database with pandas
source = pd.read_sql("SELECT * FROM dbo.A017", connection)

# Print the dataframe
print(source)



Install dask

In [ ]:
!pip3 install dask

Conect to target database

In [ ]:
import pandas as pd
import pyodbc

# Define the connection string for the target database (Oracle EBS)
target_connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=Rahul\MSSQLSERVER01;"
    "DATABASE=ORACLE_EBS_HACK;"
    "Trusted_Connection=yes;"
)

# Establish the connection
connection = pyodbc.connect(target_connection_string)

# Use the connection to query the database with pandas
source = pd.read_sql("SELECT * FROM APPLSYS_FND_FLEX_VALUES", connection)

# Print the dataframe
print(source)

# Get the Schema Infos 

In [ ]:
# Get the schema information for the Oracle EBS database
target_cursor = target_connection.cursor()
target_cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS")
target_schema = target_cursor.fetchall()

# Get the schema information for the ECC60jkl_HACK database
source_cursor = connection.cursor()
source_cursor.execute("SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS")
source_schema = source_cursor.fetchall()

In [ ]:
target_schema_dict = {}
for table, column, data_type in target_schema:
    print(f"Target schema: {table}.{column} ({data_type})")
    if table not in target_schema_dict:
        target_schema_dict[table] = []
    target_schema_dict[table].append((column, data_type))

print(f"Target schema dict: {target_schema_dict}")

source_schema_dict = {}
for table, column, data_type in source_schema:
    print(f"Source schema: {table}.{column} ({data_type})")
    if table not in source_schema_dict:
        source_schema_dict[table] = []
    source_schema_dict[table].append((column, data_type))

print(f"Source schema dict: {source_schema_dict}")

# Finding similar tables and columms

In [ ]:
similar_tables = {}
for target_table, target_columns in target_schema_dict.items():
    for source_table, source_columns in source_schema_dict.items():
        similarity = len(set(target_columns) & set(source_columns)) / len(set(target_columns) | set(source_columns))
        if similarity > 0.5:  # adjust the threshold as needed
            similar_tables[target_table] = source_table

print(f"Similar tables: {similar_tables}")

In [ ]:
similar_columns = {}
for target_table, target_columns in target_schema_dict.items():
    for source_table, source_columns in source_schema_dict.items():
        if target_table == source_table:  # only compare columns within the same table
            for target_column_name in [col[1] for col in target_columns]:
                for source_column_name in [col[1] for col in source_columns]:
                    if target_column_name == source_column_name:
                        similarity = 1.0  # exact match
                    else:
                        similarity = levenshtein_distance(target_column_name, source_column_name)
                    if similarity > 0.5:  # adjust the threshold as needed
                        similar_columns[(target_table, target_column_name)] = (source_table, source_column_name)

print(f"Similar columns: {similar_columns}")

install jellyfish 

In [ ]:
!pip3 install jellyfish 

# Mapping

In [ ]:
import jellyfish

def levenshtein_distance(s1, s2):
    return jellyfish.levenshtein_distance(s1, s2)

column_mappings = {}
for target_table, source_table in similar_tables.items():
    target_columns = target_schema_dict[target_table]
    source_columns = source_schema_dict[source_table]
    for target_column, target_data_type in target_columns:
        for source_column, source_data_type in source_columns:
            if target_data_type == source_data_type:
                similarity = 1 - levenshtein_distance(target_column, source_column) / max(len(target_column), len(source_column))
                if similarity > 0.5:  # adjust the threshold as needed
                    column_mappings[(target_table, target_column)] = (source_table, source_column)

Refining mapping

In [ ]:
import pandas as pd
from scipy import stats
import sqlite3

# Define database connections
target_connection = sqlite3.connect('target_database.db')
source_connection = sqlite3.connect('source_database.db')

target_cursor = target_connection.cursor()
source_cursor = source_connection.cursor()

# Get table names
target_tables = [table[0] for table in target_cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")]
source_tables = [table[0] for table in source_cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")]

# Initialize column mappings
column_mappings = {}
refined_column_mappings = {}

# Loop through tables and columns
for target_table in target_tables:
    for source_table in source_tables:
        target_columns = [column[1] for column in target_cursor.execute(f"PRAGMA table_info({target_table})")]
        source_columns = [column[1] for column in source_cursor.execute(f"PRAGMA table_info({source_table})")]
        for target_column in target_columns:
            for source_column in source_columns:
                if target_column == source_column:
                    column_mappings[(target_table, target_column)] = (source_table, source_column)

# Refine column mappings
for (target_table, target_column), (source_table, source_column) in column_mappings.items():
    print(f"Comparing {target_table}.{target_column} with {source_table}.{source_column}")
    target_data = pd.read_sql(f"SELECT {target_column} FROM {target_table}", target_connection)
    source_data = pd.read_sql(f"SELECT {source_column} FROM {source_table}", source_connection)
    
    print(f"Target data: {target_data.shape}")
    print(f"Source data: {source_data.shape}")
    
    # Check data distribution
    target_dist = target_data[target_column].value_counts().index
    source_dist = source_data[source_column].value_counts().index
    similarity = len(set(target_dist) & set(source_dist)) / len(set(target_dist) | set(source_dist))
    print(f"Similarity: {similarity}")
    if similarity > 0.5:
        # Check data types
        if target_data[target_column].dtype == source_data[source_column].dtype:
            print(f"Data types match: {target_data[target_column].dtype}")
            # Check statistical properties
            target_stats = target_data[target_column].describe()
            source_stats = source_data[source_column].describe()
            print(f"Target stats: {target_stats}")
            print(f"Source stats: {source_stats}")
            ks_test_p_value = stats.ks_2samp(target_data[target_column], source_data[source_column])[1]
            print(f"KS test p-value: {ks_test_p_value}")
            if ks_test_p_value > 0.05:
                refined_column_mappings[(target_table, target_column)] = (source_table, source_column)
                print(f"Added mapping: {target_table}.{target_column} -> {source_table}.{source_column}")
        else:
            print(f"Data types don't match: {target_data[target_column].dtype} vs {source_data[source_column].dtype}")
    else:
        print(f"Similarity too low: {similarity}")

print(f"Refined column mappings: {refined_column_mappings}")

Function to creat a table to display comparitions

In [ ]:
import pandas as pd

# Create a table to display the comparisons
comparisons = []
for (target_table, target_column), (source_table, source_column) in refined_column_mappings.items():
    target_data = pd.read_sql(f"SELECT {target_column} FROM {target_table}", target_connection)
    source_data = pd.read_sql(f"SELECT {source_column} FROM {source_table}", source_connection)
    
    comparisons.append({
        'Target Table': target_table,
        'Target Column': target_column,
        'Source Table': source_table,
        'Source Column': source_column,
        'Data Type': target_data[target_column].dtype,
        'Similarity': len(set(target_data[target_column].value_counts().index) & set(source_data[source_column].value_counts().index)) / len(set(target_data[target_column].value_counts().index) | set(source_data[source_column].value_counts().index)),
        'KS Test p-value': stats.ks_2samp(target_data[target_column], source_data[source_column])[1]
    })

comparisons_df = pd.DataFrame(comparisons)
print(comparisons_df)

# Value Mapping Rules

In [ ]:
value_mapping_rules = []

for mapping_rule in mapping_rules:
    source_column = mapping_rule["S_Field"]
    target_column = mapping_rule["T_Field"]
    source_data_type = mapping_rule["S_Field_Type"]
    target_data_type = mapping_rule["T_Field_Type"]

    # Handle value mapping for each data type
    if source_data_type == "string" and target_data_type == "string":
        # Handle string value mapping (e.g., harmonizing terminology discrepancies)
        value_mapping_rule = {
            "S_Field": source_column,
            "T_Field": target_column,
            "MAP_TYPE": "EQ",  # or SQL for more complex mappings
            "Transformation Syntax": "LOWER(TRIM({}))"  # example transformation syntax
        }
        value_mapping_rules.append(value_mapping_rule)

    elif source_data_type == "date" and target_data_type == "date":
        # Handle date value mapping (e.g., converting date formats)
        value_mapping_rule = {
            "S_Field": source_column,
            "T_Field": target_column,
            "MAP_TYPE": "SQL",
            "Transformation Syntax": "TO_DATE({}, 'YYYY-MM-DD')"  # example transformation syntax
        }
        value_mapping_rules.append(value_mapping_rule)

    elif source_data_type == "numeric" and target_data_type == "numeric":
        # Handle numeric value mapping (e.g., converting decimal formats)
        value_mapping_rule = {
            "S_Field": source_column,
            "T_Field": target_column,
            "MAP_TYPE": "EQ",  # or SQL for more complex mappings
            "Transformation Syntax": "ROUND({}, 2)"  # example transformation syntax
        }
        value_mapping_rules.append(value_mapping_rule)

    else:
        # Handle other data type combinations (e.g., boolean, timestamp, etc.)
        pass

print(f"Value mapping rules: {value_mapping_rules}")